In [4]:
!pip install requests beautifulsoup4 openai langchain-openai

In [6]:
import requests
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI
import time

def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extrair apenas o conteúdo principal do artigo
        article_content = soup.find('article')

        if article_content:
            # Remover seções indesejadas
            comments_section = article_content.find('section', {'id': 'comments'})
            if comments_section:
                comments_section.decompose()

            hide_comments_modal = article_content.find('div', {'id': 'hide-comments-modal'})
            if hide_comments_modal:
                hide_comments_modal.decompose()

            # Remover scripts e estilos
            for script_or_style in article_content(['script', 'style']):
                script_or_style.decompose()

            texto = article_content.get_text(separator=' ')

            # Limpar o texto
            lines = (line.strip() for line in texto.splitlines())
            parts = (phrase.strip() for line in lines for phrase in line.split("  "))
            texto_limpo = '\n'.join(part for part in parts if part)
            print(texto_limpo)
            return texto_limpo
        else:
            print("Article content not found.")
            return None
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")
        return None

client = AzureChatOpenAI(
    azure_endpoint="YOUR_AZURE_ENDPOINT",
    api_key="YOUR_API_KEY",
    api_version="YOUR_API_VERSION",
    azure_deployment="YOUR_DEPLOYMENT_NAME",
    max_retries=0
)

def translate_text_in_chunks(text, lang):
    max_tokens_per_request = 500  # Ajuste conforme necessário
    text_chunks = [text[i:i+max_tokens_per_request] for i in range(0, len(text), max_tokens_per_request)]

    translated_text = ""
    for chunk in text_chunks:
        messages = [
            ("system", "Você atua como tradutor de textos"),
            ("user", f"Traduza o seguinte trecho para o idioma {lang}:\n\n{chunk}")
        ]

        try:
            response = client.invoke(messages)
            translated_text += response.content + "\n\n"
        except Exception as e:
            print("Erro ao chamar a API:", e)
            print("Aguardando antes de tentar novamente...")
            time.sleep(60)

    return translated_text

# URL do artigo
url = 'https://dev.to/esdanielgomez/using-azure-openai-service-gpt-4-in-net-2l1f'
text = extract_text_from_url(url)

if text:
    article = translate_text_in_chunks(text, "pt-br")
    print(article)


Daniel Gomez
Posted on
Jan 30
Using Azure OpenAI Service (GPT-4) in .NET
# azure
# dotnet
# openai
# chatgpt
Hey, there! In this tutorial, we'll see how to call an Azure OpenAI Service GPT model from C#.
Required data from Azure:
Key and endpoint
- We can find this information in the
Keys and endpoint
section in our Azure OpenAI Service resource.
Deployment name
- This would be the instance of the model to use. In this case, we can go to the Azure OpenAI Service studio, and in the
Deployments
section specify a new one:
Azure OpenAI client library for .NET
Azure OpenAI client library for .NET is an adaptation of OpenAI's REST APIs that provides an idiomatic interface and rich integration with the rest of the Azure SDK ecosystem.
This library will allow us to use the GPT model from C#.
NuGet package:
Azure.AI.OpenAI
dotnet add package Azure.AI.OpenAI --version 1.0.0-beta.12
Enter fullscreen mode
Exit fullscreen mode
Code in C#
With the NuGet package installed, this is a C# class that wil

KeyboardInterrupt: 